In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    #y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
!ls "/content/drive/My Drive/Colab/"

20_30_weights.h5		 unique_chars_utf.npy  y_20_30.npy
bangla_speech_recognition.ipynb  utt_spk_text.tsv      y_len_20_30.npy
u_chars_utf.npy			 x_20_30.npy	       y_mapped_utf_20_30.npy


In [ ]:
import numpy as np
from keras import regularizers
x=np.load("/content/drive/My Drive/Colab/test_x_0_10.npy")

Using TensorFlow backend.


In [ ]:
y=np.load("/content/drive/My Drive/Colab/test_y_0_10.npy")


In [ ]:
y_utf=np.load("/content/drive/My Drive/Colab/test_x_mapped0_10.npy")
y_len=np.load("/content/drive/My Drive/Colab/test_y_len_0_10.npy")
#y=np.load("/content/drive/My Drive/Colab/y_0_10_78utf_spec.npy")

In [ ]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Lambda
from keras.layers import Dropout
from keras.regularizers import l2
from keras.initializers import random_normal
from keras.utils.conv_utils import conv_output_length
from keras.layers import GaussianNoise

import numpy as np

from keras import backend as K
from keras.models import Model, Sequential
from keras.layers.recurrent import SimpleRNN
from keras.layers import Dense, Activation, Bidirectional, Reshape,Flatten, Lambda, Input,\
    Masking, Convolution1D, BatchNormalization, GRU, Conv1D, RepeatVector, Conv2D
from keras.optimizers import SGD, adam
from keras.layers import ZeroPadding1D, Convolution1D, ZeroPadding2D, Convolution2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import TimeDistributed, Dropout
from keras.layers.merge import add  # , # concatenate BAD FOR COREML
from keras.utils.conv_utils import conv_output_length
from keras.activations import relu
import numpy as np 
import keras
from keras.layers import *
from keras.models import Sequential
from keras.callbacks import TensorBoard
from time import time
from keras.models import Model
import os
import tensorflow as tf
from keras.models import load_model
#from pre_processing_v4 import *
from sklearn.model_selection import train_test_split
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
size=x.shape[0]
x.shape

(2000, 5000)

In [ ]:
input_length = np.zeros([size, 1])
label_length = np.zeros([size, 1])
audio_labels = np.ones([size, 30])




inputs = {'the_input': x,
          'the_labels': audio_labels,
          'input_length': input_length,
          'label_length': label_length,
          }


outputs = {'ctc': np.zeros([size])}


input_length2=np.zeros([size, 1])
label_length2=np.zeros([size, 1])

for i in range(size):
	input_length2[i] = 5000
	#label_length2[i]=(i+100)/5

label_length2 = y_len
print(label_length2)

[10 10 10 ... 10  6  6]


In [ ]:
x= x.reshape(x.shape[0],5000,1)
input_shape=(5000,1)

In [ ]:
x.shape

(2000, 5000, 1)

In [ ]:
def selu(x):
    # from Keras 2.0.6 - does not exist in 2.0.4
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
       x: A tensor or variable to compute the activation function for.
    # References
       - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * K.elu(x, alpha)


def clipped_relu(x):
    return relu(x, max_value=20)

In [ ]:
rnn_size=100
fc_size=248
output_dim=79

input_length2=np.zeros([size, 1])
label_length2=np.zeros([size, 1])

for i in range(size):
	input_length2[i] = 5000
	#label_length2[i]=(i+100)/5

label_length2 = y_len
print(label_length2)


from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({"clipped_relu": clipped_relu})    
K.set_learning_phase(1)

input_shape=(5000,1)

init = random_normal(stddev=0.046875)

input_data = Input(name='the_input', shape=input_shape, dtype='float32')
q = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(input_data)
conv = ZeroPadding1D(padding=(0, 2048))(q)

for l in range(1):
	q = Conv1D(filters=fc_size, name='conv_{}'.format(l+1), kernel_size=11, padding='valid', activation='relu', strides=2)(conv)

q = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(q)

for l in range(1):
	q = Bidirectional(GRU(rnn_size, name='fc_{}'.format(l + 1), return_sequences=True, activation='relu', kernel_initializer='glorot_uniform'),
                      merge_mode='sum')(q)

q = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(q)

q = TimeDistributed(Dense(fc_size, activation=clipped_relu))(q)
y_pred = TimeDistributed(Dense(output_dim, name="y_pred", activation="softmax"))(q)

model1=Model(inputs=input_data, outputs=y_pred)

labels = Input(name='the_labels',
                   shape=[30], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')



loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred,
                                                                       labels,
                                                                       input_length,
                                                                       label_length])

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)


[17 21 13 ... 10  8  8]


In [ ]:
def clipped_relu(x):
    return relu(x, max_value=20)

In [ ]:
rnn_size=100
fc_size=1048
output_dim=79


  
  
init = random_normal(stddev=0.046875)

input_data = Input(name='the_input', shape=input_shape, dtype='float32')
#q = Conv1D(64, kernel_size=3, activation='relu', input_shape=(10,22))(input_data)

q = Bidirectional(LSTM(rnn_size, return_sequences=True, 
                                kernel_initializer='he_normal', name='birnn'), merge_mode='sum')(input_data)


y_pred = TimeDistributed(Dense(output_dim, name="y_pred", kernel_initializer=init, bias_initializer=init, activation="softmax"))(q)

model1=Model(inputs=input_data, outputs=y_pred)

labels = Input(name='the_labels', shape=[10], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')



loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred,
                                                                       labels,
                                                                       input_length,
                                                                       label_length])

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
#model.load_weights('short_data_v8.h5')
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='sgd',metrics=['accuracy'])

outputs = {'ctc': np.zeros([len(x)])}

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=5, min_lr=0.0001)

In [ ]:
#model.load_weights('/content/drive/My Drive/Colab/0_10_78utf_spec.h5')
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam',metrics=['accuracy'])

outputs = {'ctc': np.zeros([len(x)])}

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=5, min_lr=0.001)

model.fit([np.array(x), np.array(y_utf), np.array(input_length2), np.array(label_length2)], outputs,epochs=100, callbacks=[reduce_lr],batch_size=200)
model.save_weights('/content/drive/My Drive/Colab/0_30_78utf_22.h5')

Epoch 1/100
55299/55299 [==============================] - 1512s 27ms/step - loss: 63.8914 - acc: 0.0000e+00
Epoch 2/100
55299/55299 [==============================] - 1493s 27ms/step - loss: 57.6870 - acc: 0.0000e+00
Epoch 3/100
55299/55299 [==============================] - 1496s 27ms/step - loss: 56.5166 - acc: 0.0000e+00
Epoch 4/100
55299/55299 [==============================] - 1492s 27ms/step - loss: 55.7958 - acc: 0.0000e+00
Epoch 5/100
55299/55299 [==============================] - 1496s 27ms/step - loss: 55.4801 - acc: 0.0000e+00
Epoch 6/100
55299/55299 [==============================] - 1496s 27ms/step - loss: 55.4003 - acc: 0.0000e+00
Epoch 7/100
55299/55299 [==============================] - 1497s 27ms/step - loss: 55.3719 - acc: 0.0000e+00
Epoch 8/100
55299/55299 [==============================] - 1496s 27ms/step - loss: 57.2969 - acc: 0.0000e+00
Epoch 9/100
55299/55299 [==============================] - 1498s 27ms/step - loss: 55.6961 - acc: 0.0000e+00
Epoch 10/100
55299/

In [1]:
# model.fit([np.array(x), np.array(y_utf), np.array(input_length2), np.array(label_length2)], outputs,epochs=3500, batch_size=200, callbacks=[reduce_lr])
